<a href="https://colab.research.google.com/github/UKJaagadhep/Data-science-and-machine-learning/blob/main/Neural_Machine_Translation/English_to_French_Neural_Machine_Translation_with_T5_(HuggingFace)_SKELETON.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.9 MB/s eta 0:00:00


In [155]:
pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 2.3 MB/s eta 0:00:00


In [37]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from datasets import load_dataset
from transformers import T5TokenizerFast
from transformers import TFAutoModelForSeq2SeqLM
from transformers import DataCollatorForSeq2Seq
from transformers import create_optimizer
import evaluate

# **DATA PREPARATION**

In [4]:
!wget https://www.manythings.org/anki/fra-eng.zip
!unzip "/content/fra-eng.zip" -d "/content/dataset/"

--2024-05-29 01:14:10--  https://www.manythings.org/anki/fra-eng.zip
Resolving www.manythings.org (www.manythings.org)... 173.254.30.110
Connecting to www.manythings.org (www.manythings.org)|173.254.30.110|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7943074 (7.6M) [application/zip]
Saving to: ‘fra-eng.zip’

fra-eng.zip         100%[===================>]   7.57M  5.74MB/s    in 1.3s    

2024-05-29 01:14:12 (5.74 MB/s) - ‘fra-eng.zip’ saved [7943074/7943074]

Archive:  /content/fra-eng.zip
  inflating: /content/dataset/_about.txt  
  inflating: /content/dataset/fra.txt  


In [7]:
dataset = load_dataset('text', data_files = '/content/dataset/fra.txt')

In [8]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 232736
    })
})

In [9]:
dataset['train'][100000]

{'text': 'Do you mind if I tag along?\tPuis-je venir avec toi ?\tCC-BY 2.0 (France) Attribution: tatoeba.org #4177437 (Hybrid) & #7499044 (Micsmithel)'}

In [10]:
dataset['train'][100000]['text']

'Do you mind if I tag along?\tPuis-je venir avec toi ?\tCC-BY 2.0 (France) Attribution: tatoeba.org #4177437 (Hybrid) & #7499044 (Micsmithel)'

In [11]:
dataset['train'][100000]['text'].split('\t')

['Do you mind if I tag along?',
 'Puis-je venir avec toi ?',
 'CC-BY 2.0 (France) Attribution: tatoeba.org #4177437 (Hybrid) & #7499044 (Micsmithel)']

In [12]:
dataset['train'][100000]['text'].split('\t')[:-1]

['Do you mind if I tag along?', 'Puis-je venir avec toi ?']

In [13]:
model_id = "t5-small"
tokenizer = T5TokenizerFast.from_pretrained(model_id)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [15]:
prefix = "translate English to French: " #We add prefix to all T5 Tokenizer inputs to denote the task

In [115]:
def preprocess_function(examples):
  inputs = [prefix + example.split('\t')[0] for example in examples['text']]
  targets = [example.split('\t')[1] for example in examples['text']]

  model_inputs = tokenizer(inputs, text_target = targets, max_length = 128, truncation = True)

  return model_inputs

In [116]:
tokenized_dataset = dataset.map(preprocess_function, batched = True)

Map:   0%|          | 0/232736 [00:00<?, ? examples/s]

In [117]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 232736
    })
})

In [118]:
tokenized_dataset['train'][10101]
#13959, 1566, 12, 2379, 10 denotes the prefix, then comes the input (english text) token input_ids and at last comes the output (french text) token input_ids
#input_ids are generally longer than the label here

{'text': 'Who hates you?\tQui te déteste ?\tCC-BY 2.0 (France) Attribution: tatoeba.org #2254852 (CK) & #2804498 (Rockaround)',
 'input_ids': [13959, 1566, 12, 2379, 10, 2645, 5591, 7, 25, 58, 1],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'labels': [6590, 3, 17, 15, 1136, 17, 2613, 3, 58, 1]}

In [119]:
cleaned_dataset = tokenized_dataset.remove_columns("text")

In [120]:
cleaned_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 232736
    })
})

In [121]:
cleaned_dataset['train'][10101]

{'input_ids': [13959, 1566, 12, 2379, 10, 2645, 5591, 7, 25, 58, 1],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'labels': [6590, 3, 17, 15, 1136, 17, 2613, 3, 58, 1]}

In [122]:
model = TFAutoModelForSeq2SeqLM.from_pretrained(model_id)
data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer, model = model, return_tensors = "tf")

All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [151]:
batch_size = 64

In [ ]:
tf_train_set = cleaned_dataset["train"].to_tf_dataset(
    shuffle = True,
    batch_size = batch_size,
    collate_fn = data_collator,
)

In [126]:
train_data = tf_train_set.take(int(0.9 * len(tf_train_set)))
val_data = tf_train_set.skip(int(0.9 * len(tf_train_set)))

In [ ]:
for i in val_data.take(1):
  print(i)

# **MODELING**

In [82]:
model.summary()

Model: "tft5_for_conditional_generation"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 shared (Embedding)          multiple                  16449536  
                                                                 
 encoder (TFT5MainLayer)     multiple                  35330816  
                                                                 
 decoder (TFT5MainLayer)     multiple                  41625344  
                                                                 
Total params: 60506624 (230.81 MB)
Trainable params: 60506624 (230.81 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


# **TRAINING**

In [ ]:
num_epochs = 6
num_train_steps = len(train_data) * num_epochs

In [ ]:
optimizer, schedule = create_optimizer(
    init_lr = 2e-5,
    num_warmup_steps = 0,
    num_train_steps = num_train_steps,
)

In [ ]:
model.compile(optimizer = optimizer)

In [ ]:
history = model.fit(train_data,
                    validation_data = val_data,
                    epochs = num_epochs)

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model_loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc = 'upper left')
plt.show()

# **EVALUATION**

In [156]:
metric = evaluate.load("sacrebleu")

In [ ]:
all_preds = []
all_labels = []

for batch in val_data:
  predictions = model.generate(
      input_ids = batch["input_ids"], attention_mask = batch["attention_mask"]
  )
  decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens = True)
  labels = batch["labels"].numpy()
  labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
  decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens = True)
  all_preds.extend(decoded_preds)
  all_labels.extend(decoded_labels)

result = metric.compute(predictions = all_preds, references = all_labels)
print(result)

In [ ]:
decoded_preds

In [ ]:
decoded_labels

# **TESTING**

In [ ]:
input_text = "Have you ever played soccer under the rain, with your friends? "
input_text = "Google Translate is a multilingual neural machine translation service developed by Google to translate text, documents and websites from one language into another."
input_text = prefix + input_text
tokenized = tokenizer([input_text], return_tensors = 'tf')
out = model.generate(**tokenized, max_length = 128)
print(out)

In [ ]:
print(tokenizer.decode(out[0], skip_special_tokens = True))

# **TESTING ORIGINAL MODEL (NO FINETUNING)**

In [153]:
original_model = TFAutoModelForSeq2SeqLM.from_pretrained(model_id)

All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [ ]:
input_text = "Have you ever played soccer under the rain, with your friends? "
input_text = "Google Translate is a multilingual neural machine translation service developed by Google to translate text, documents and websites from one language into another."
input_text = prefix + input_text
tokenized = tokenizer([input_text], return_tensors = 'tf')
out = original_model.generate(**tokenized, max_length = 128)
print(out)

In [ ]:
print(tokenizer.decode(out[0], skip_special_tokens = True))